<a href="https://colab.research.google.com/github/Blusooyeon/bigdata-analysist-certificate/blob/main/practice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 작업형 1유형

데이터 출처 : https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset (후처리 작업)              

데이터 설명 : 뇌졸증 발생여부 예측
dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke/train.csv

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


In [4]:
#문제1. 성별이 Male인 환자들의 age의 평균값은 ?
df['age'] = df['age'].str.replace('*','').astype('int')
df[df.gender == 'Male'].age.mean()

<ipython-input-4-384a2145baa2>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['age'] = df['age'].str.replace('*','').astype('int')


44.68623481781376

In [6]:
#문제2 . bmi컬럼의 결측치를 bmi컬럼의 결측치를 제외한 나머지 값들의 중앙값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라
df.bmi.fillna(df.bmi.median())
round(df.bmi.mean(),3)

29.198

In [8]:
#문제3. bmi컬럼의 각 결측치들을 직전의 행의 bmi값으로 채웠을 경우 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라
df1 = df.bmi.fillna(method = 'ffill')
round(df1.mean(),3)

29.188

In [9]:
#문제4. bmi컬럼의 각 결측치들을 결측치를 가진 환자 나이대(10단위)의 평균 bmi 값으로 대체한 후 대체된 bmi 컬럼의 평균을 소숫점 이하 3자리 까지 구하여라

# 결측치 제외 나이대별 평균값 계산 및 dictionary 형태로 변환
mean = df[df.bmi.notnull()].groupby(df.age//10 *10).bmi.mean()
dic = { x:y for x,y in mean.items()}
idx =df.loc[df.bmi.isnull(),['age','bmi']].index

In [10]:
# 결측치 매핑
df.loc[df.bmi.isnull(),'bmi']  =(df[df.bmi.isnull()].age//10*10).map(lambda x : dic[x])
df.bmi.mean()

29.2627029367386

In [12]:
#문제5. avg_glucose_level 컬럼의 값이 200이상인 데이터를 모두 199로 변경하고 stroke값이 1인 데이터의 avg_glucose_level값의 평균을 소수점이하 3자리 까지 구하여라
df.loc[df.avg_glucose_level >=200,'avg_glucose_level'] =199
round(df[df.stroke == 1].avg_glucose_level.mean(),3)

125.188

#### 작업 2유형
데이터 출처 : https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset (후처리 작업)       
데이터 설명 : 뇌졸증 발생여부 예측

In [13]:
import pandas as pd
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/test.csv')

display(train.head())
display(test.head())

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,47472,Female,58,0,0,Yes,Private,Urban,107.26,38.6,formerly smoked
1,36841,Male,78,1,0,Yes,Self-employed,Rural,56.11,25.5,formerly smoked
2,3135,Female,73,0,0,No,Self-employed,Rural,69.35,NaN,never smoked
3,65218,Male,2,0,0,No,children,Rural,109.10,20.0,Unknown
4,1847,Female,20,0,0,No,Govt_job,Rural,79.53,NaN,never smoked


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# 데이터 전처리

train['age'] =train['age'].str.replace('*','').astype('int')
train['bmi'] = train['bmi'].fillna(train['bmi'].mean())
test['bmi'] = test['bmi'].fillna(test['bmi'].mean())
x = train.drop(columns =['id','stroke'])
xd = pd.get_dummies(x)
y = train['stroke']


<ipython-input-14-88378b975bfb>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['age'] =train['age'].str.replace('*','').astype('int')


In [15]:
x_train,x_test,y_train,y_test = train_test_split(xd,y,stratify =y ,random_state=1)
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
pred = rf.predict_proba(x_test)
print('test roc score : ',roc_auc_score(y_test,pred[:,1]))

test roc score :  0.8542301382704476


In [16]:
# one-hot encoding시 train셋에만 존재하는 컬럼이 존재

test_preprocessing =pd.get_dummies(test.drop(columns=['id']))
test_preprocessing[list(set(x_train.columns) -set(test_preprocessing))] =0
test_preprocessing =test_preprocessing[x_train.columns]
test_pred = rf.predict_proba(test_preprocessing)

In [17]:
pd.DataFrame({'id': test.id, 'stroke': test_pred[:,1]}).to_csv('003000000.csv', index=False)
